<a href="https://colab.research.google.com/github/Thanos002/IL_Measurement/blob/main/IL_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
def Extract_data (query,tablename):

  #url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
  frames = []
  skip = 0
  allfound = false
  while(!allfound):
    tmp_query = query%(skip)
    r = requests.post(url, json={'query': tmp_query})
    json_data = json.loads(r.text)
    try:
      df_data = json_data['data'][tablename]
      df = pd.DataFrame(df_data)
      frames.append(df)
      skip += 1000
      if(df_data < 1000):
        allfound = true
    except KeyError:
      break

  result = pd.concat(frames)
  return result


In [ ]:
class Token:
  def __init__(self, name, id, type):
    self.name = name
    self.id = id
    self.type = type

In [ ]:
class Pool:
  def __init__(self, id, fee, token1, token2):
    self.id = id
    self.group = token1.type + "/" + token2.type
    self.fee = fee
    self.token1 = token1
    self.token2 = token2

In [ ]:
class PoolContainer:
    def __init__(self):
        self.pools = []

    def add_pool(self, pool):
        self.pools.append(pool)

    def __iter__(self):
        return iter(self.pools)

In [ ]:
#Tokens
dai = Token("DAI","0x6b175474e89094c44da98b954eedeac495271d0f", "stable")
usdc = Token("USDC", "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "stable")
weth = Token("WETH","0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2", "risky")
mkr = Token("MKR","0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2", "risky")
wbtc = Token("WBTC","0x2260fac5e5542a773aa44fbcfedf7c193bc2c599", "risky")

In [ ]:
# Stable pools:
dai_usdc_500 = Pool("0x6c6bc977e13df9b0de53b251522280bb72383700", 500, dai, usdc)
dai_usdc_100 = Pool("0x5777d92f208679db4b9778590fa3cab3ac9e2168", 100, dai, usdc)

# Stable-risky pools
usdc_weth_10000 = Pool("0x7bea39867e4169dbe237d55c8242a8f2fcdcc387", 10000, usdc, weth)
usdc_weth_3000 = Pool("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", 3000, usdc, weth)
usdc_weth_500 = Pool("0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640", 500, usdc, weth)
usdc_weth_100 = Pool("0xe0554a476a092703abdb3ef35c80e0d76d32939f", 100, usdc, weth)

#Risky pools
mkr_weth_10000 = Pool("0x3afdc5e6dfc0b0a507a8e023c9dce2cafc310316", 10000, mkr, weth)
wbtc_weth_3000 = Pool("0xcbcdf9626bc03e24f779434178a73a0b4bad62ed", 3000, wbtc, weth)
wbtc_weth_500 = Pool("0x4585fe77225b41b697c938b018e2ac67ac5a20c0", 500, wbtc, weth)


In [ ]:
pools_container = PoolContainer()
pools_container.add_pool(dai_usdc_500)
pools_container.add_pool(dai_usdc_100)
pools_container.add_pool(usdc_weth_10000)
pools_container.add_pool(usdc_weth_3000)
pools_container.add_pool(usdc_weth_500)
pools_container.add_pool(usdc_weth_100)
pools_container.add_pool(mkr_weth_10000)
pools_container.add_pool(wbtc_weth_3000)
pools_container.add_pool(wbtc_weth_500)

In [ ]:
for pool in pools_container:
    print(pool.token1.name + "/" + pool.token2.name + "; " + str(pool.fee))

DAI/USDC; 500
DAI/USDC; 100
USDC/WETH; 10000
USDC/WETH; 3000
USDC/WETH; 500
USDC/WETH; 100
MKR/WETH; 10000
WBTC/WETH; 3000
WBTC/WETH; 500


In [ ]:

ZERO = 0
Q128 = 2 ** 128
Q256 = 2 ** 256

def subIn256(x, y):
    difference = x - y
    if difference < ZERO:
        return Q256 + difference
    else:
        return difference

def toBigNumber(value):
    return int(value)  # Convert string or number to Python integer, in case large number is in strign format

def getFees(feeGrowthGlobal0, feeGrowthGlobal1, feeGrowth0Low, feeGrowth0Hi, feeGrowthInside0, feeGrowth1Low, feeGrowth1Hi, feeGrowthInside1, liquidity, decimals0, decimals1, tickLower, tickUpper, tickCurrent):
    feeGrowthGlobal_0 = toBigNumber(feeGrowthGlobal0)
    feeGrowthGlobal_1 = toBigNumber(feeGrowthGlobal1)
    tickLowerFeeGrowthOutside_0 = toBigNumber(feeGrowth0Low)
    tickLowerFeeGrowthOutside_1 = toBigNumber(feeGrowth1Low)
    tickUpperFeeGrowthOutside_0 = toBigNumber(feeGrowth0Hi)
    tickUpperFeeGrowthOutside_1 = toBigNumber(feeGrowth1Hi)
    tickLowerFeeGrowthBelow_0 = ZERO
    tickLowerFeeGrowthBelow_1 = ZERO
    tickUpperFeeGrowthAbove_0 = ZERO
    tickUpperFeeGrowthAbove_1 = ZERO

    if tickCurrent >= tickUpper:
        tickUpperFeeGrowthAbove_0 = subIn256(feeGrowthGlobal_0, tickUpperFeeGrowthOutside_0)
        tickUpperFeeGrowthAbove_1 = subIn256(feeGrowthGlobal_1, tickUpperFeeGrowthOutside_1)
    else:
        tickUpperFeeGrowthAbove_0 = tickUpperFeeGrowthOutside_0
        tickUpperFeeGrowthAbove_1 = tickUpperFeeGrowthOutside_1

    if tickCurrent >= tickLower:
        tickLowerFeeGrowthBelow_0 = tickLowerFeeGrowthOutside_0
        tickLowerFeeGrowthBelow_1 = tickLowerFeeGrowthOutside_1
    else:
        tickLowerFeeGrowthBelow_0 = subIn256(feeGrowthGlobal_0, tickLowerFeeGrowthOutside_0)
        tickLowerFeeGrowthBelow_1 = subIn256(feeGrowthGlobal_1, tickLowerFeeGrowthOutside_1)

    fr_t1_0 = subIn256(subIn256(feeGrowthGlobal_0, tickLowerFeeGrowthBelow_0), tickUpperFeeGrowthAbove_0)
    fr_t1_1 = subIn256(subIn256(feeGrowthGlobal_1, tickLowerFeeGrowthBelow_1), tickUpperFeeGrowthAbove_1)
    feeGrowthInsideLast_0 = toBigNumber(feeGrowthInside0)
    feeGrowthInsideLast_1 = toBigNumber(feeGrowthInside1)

    uncollectedFees_0 = (liquidity * subIn256(fr_t1_0, feeGrowthInsideLast_0)) // Q128
    uncollectedFees_1 = (liquidity * subIn256(fr_t1_1, feeGrowthInsideLast_1)) // Q128

    uncollectedFeesAdjusted_0 = uncollectedFees_0 / (10 ** decimals0)
    uncollectedFeesAdjusted_1 = uncollectedFees_1 / (10 ** decimals1)
    print(f"Amount fees token 0 in lowest decimal: {int(uncollectedFees_0)}")
    print(f"Amount fees token 1 in lowest decimal: {int(uncollectedFees_1)}")
    print(f"Amount fees token 0 Human format: {uncollectedFeesAdjusted_0:.{decimals0}f}")
    print(f"Amount fees token 1 Human format: {uncollectedFeesAdjusted_1:.{decimals1}f}")


In [ ]:
getFees(3094836483914812667943230173936420,1,37180414779992829129391081655145,233371140530963296710329726203514,0,1,1,1,10860507277202,6,6,192180,193380,201780)

Amount fees token 0 in lowest decimal: 6261655
Amount fees token 1 in lowest decimal: 3695639122248373304231208116609338612196379944026111
Amount fees token 0 Human format: 6.261655
Amount fees token 1 Human format: 3695639122248373590578266900963621657483673600.000000


In [ ]:
class LiquidityProvider:
    def __init__(self, address):
        """
        Initializes a new LiquidityProvider instance.

        Parameters:
        address (str): The Ethereum address of the liquidity provider.
        """
        self.address = address
        self.positions = []

    def add_position(self, position):
        """
        Adds a new position to the liquidity provider's list of positions.

        Parameters:
        position (Position): The position to be added to this provider.
        """
        self.positions.append(position)
        position.provider = self  # Establish the reverse link from Position to LiquidityProvider

    def get_positions(self):
        """
        Returns a list of all positions for this liquidity provider.
        """
        return self.positions

    def __str__(self):
        """
        Returns a string representation of the LiquidityProvider object.
        """
        return f"Liquidity Provider Address: {self.address}"

class Event:
    def __init__(self, block_number, amount):
        """
        Initializes a new Event instance.

        Parameters:
        block_number (int): The block number at which the event occurred.
        amount (float): The amount of liquidity involved in the event.
        """
        self.block_number = block_number
        self.amount = amount

    def __str__(self):
        """
        Return a basic representation of the event, should be overridden by subclasses.
        """
        return f"Event at Block {self.block_number}, Amount: {self.amount}"

class Mint(Event):
    def __str__(self):
        return f"Mint: Block {self.block_number}, Amount: {self.amount}"

class Burn(Event):
    def __str__(self):
        return f"Burn: Block {self.block_number}, Amount: {self.amount}"

class Add(Event):
    def __str__(self):
        return f"Add: Block {self.block_number}, Amount: {self.amount}"

class Remove(Event):
    def __str__(self):
        return f"Remove: Block {self.block_number}, Amount: {self.amount}"

class Position:
    def __init__(self, provider, events):
        """
        Initializes a new Position instance.

        Parameters:
        provider (LiquidityProvider): The liquidity provider associated with this position.
        events (list of Event): A list of events associated with this position.
        """
        assert len(events) >= 2, "There must be at least two events for a position."
        self.collectedFeesToken0
        self.provider = provider
        self.events = events

    def __str__(self):
        events_str = "\n".join([str(event) for event in self.events])
        return f"Position for {self.provider}\nEvents:\n{events_str}"
